In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Thêm đường dẫn đến thư mục src để import các module
sys.path.append(os.path.abspath(os.path.join('..')))

from src import data_processing as dp
from src import visualization as viz

# Cấu hình để hiển thị biểu đồ trong notebook
%matplotlib inline

# Tải dữ liệu
filepath = '../data/raw/aug_train.csv'
data = dp.load_data(filepath)

# 1. Tổng Quan Dữ Liệu

### 1.1. Thông tin cơ bản

In [ ]:
basic_info = dp.get_basic_info(data)
print(f"Số dòng: {basic_info['shape'][0]}")
print(f"Số cột: {basic_info['shape'][1]}")
print(f"Kích thước tổng thể: {basic_info['memory_usage'] / 1e6:.2f} MB")

**Mỗi dòng trong tập dữ liệu này đại diện cho một ứng viên đã đăng ký tham gia chương trình đào tạo của công ty.**
Dòng dữ liệu mô tả đầy đủ hồ sơ của từng ứng viên, bao gồm thông tin nhân khẩu học, trình độ học vấn, kinh nghiệm làm việc, tình trạng công việc hiện tại, số giờ đào tạo và một biến mục tiêu cho biết họ có đang tìm kiếm cơ hội việc làm mới hay có khả năng gắn bó làm việc cho công ty sau khi hoàn thành khóa đào tạo.


In [ ]:
# Hiển thị 5 dòng đầu tiên
print("5 dòng đầu tiên:")
print(data[:5])

# Hiển thị 5 dòng cuối cùng
print("\n5 dòng cuối cùng:")
print(data[-5:])

### 1.2. Tính toàn vẹn của dữ liệu

In [ ]:
duplicate_info = dp.check_duplicates(data)
print(f"Số dòng trùng lặp (không tính enrollee_id): {duplicate_info['duplicated_rows']}")
print(f"Tỷ lệ trùng lặp: {duplicate_info['duplicated_percentage']:.2f}%")

In [ ]:
# Hiển thị các dòng trùng lặp nếu có
if duplicate_info['duplicated_rows'] > 0:
    print("\nCác dòng được xác định là trùng lặp:")
    for duplicate in duplicate_info['representative_duplicates']:
        print(duplicate['row'])
        print(f"Count: {duplicate['count']}")
        print()
else:
    print("\nKhông tìm thấy dòng nào trùng lặp.")

**Xử lý dữ liệu trùng lặp:**

Các dòng trùng lặp **nên được loại bỏ**.

Toàn bộ 49 bản ghi trùng đều xuất hiện **dưới dạng cặp giống hệt nhau trên tất cả các thuộc tính trừ `enrollee_id`**. Điều này cho thấy những bản ghi này thực chất mô tả **cùng một ứng viên** được ghi nhận nhiều lần, không mang thêm thông tin mới.

Tỷ lệ trùng lặp tuy nhỏ (**0.26%**) nhưng việc loại bỏ vẫn cần thiết để đảm bảo dữ liệu **sạch**, tránh tác động tiêu cực đến mô hình dự báo.

In [ ]:
# Có bao nhiêu dòng hoàn toàn đủ/trống thông tin
row_completeness = dp.count_row_completeness(data)
print(f"Số dòng đủ hết thông tin: {row_completeness['complete_rows']}")
print(f"Số dòng thiếu hết thông tin: {row_completeness['empty_rows']}")

### 1.3. Thông tin cột và kiểu dữ liệu

**Ý nghĩa của từng cột:**

- **enrollee_id**: ID duy nhất cho ứng viên.
- **city**: Mã thành phố.
- **city_development_index**: Chỉ số phát triển của thành phố (đã được scale).
- **gender**: Giới tính của ứng viên.
- **relevent_experience**: Kinh nghiệm liên quan của ứng viên.
- **enrolled_university**: Loại khóa học đại học đã đăng ký (nếu có).
- **education_level**: Trình độ học vấn của ứng viên.
- **major_discipline**: Chuyên ngành học chính của ứng viên.
- **experience**: Tổng số năm kinh nghiệm.
- **company_size**: Số lượng nhân viên trong công ty của nhà tuyển dụng hiện tại.
- **company_type**: Loại hình của nhà tuyển dụng hiện tại.
- **last_new_job**: Chênh lệch số năm giữa công việc trước đó và công việc hiện tại.
- **training_hours**: Số giờ đào tạo đã hoàn thành.
- **target**: Biến mục tiêu.
  - `0`: Không tìm kiếm sự thay đổi công việc.
  - `1`: Đang tìm kiếm sự thay đổi công việc.

**Các cột có liên quan và các cột cần loại bỏ:**

- **Cột cần loại bỏ:** `enrollee_id` là một mã định danh duy nhất và không có giá trị dự đoán. Nó nên được loại bỏ khỏi quá trình huấn luyện mô hình.
- **Các cột liên quan:** Tất cả các cột khác đều có khả năng chứa thông tin hữu ích để dự đoán biến mục tiêu `target`.

In [ ]:
print("Kiểu dữ liệu của các cột:")
for name, dtype in basic_info['dtypes'].items():
    print(f"- {name}: {dtype}")

**Ghi chú kiểu dữ liệu `<U...`:**

Khi đọc dữ liệu từ file CSV bằng **NumPy** (ví dụ dùng `np.loadtxt` hoặc `np.genfromtxt`), các cột chứa dữ liệu dạng chữ sẽ được NumPy chuyển thành kiểu **chuỗi Unicode** với định dạng **`<U...`**. Ký hiệu này thể hiện kiểu *Unicode string* có độ dài cố định, trong đó số phía sau (ví dụ `<U8`, `<U23`, ...) là **độ dài tối đa của chuỗi trong cột**. Khi NumPy gặp cột chứa ký tự, nó không có kiểu linh hoạt như `object` của pandas, do đó nó buộc phải chọn một kiểu chuỗi Unicode có độ dài đủ lớn để chứa giá trị dài nhất trong cột. 

# 2. Phân Tích Giá Trị Thiếu

### 2.1. Đánh giá tổng quan 

In [ ]:
missing_summary = dp.get_missing_summary(data)
print("Tóm tắt giá trị thiếu theo từng cột:")
for col, summary in missing_summary.items():
    if summary['missing_count'] > 0:
        print(f"- Cột '{col}': {summary['missing_count']} giá trị thiếu ({summary['missing_percentage']:.2f}%)")

In [ ]:
viz.plot_missing_summary(missing_summary)

**Nhận xét:**

**Missing values không random mà có pattern:**
- Các biến liên quan đến **công ty hiện tại** có tỷ lệ thiếu cao nhất và gần như tương đương nhau: `company_type` (32.05%) và `company_size` (30.99%), cho thấy chúng thường thiếu đồng thời
- Biến `gender` (23.53%) và `major_discipline` (14.68%) có tỷ lệ thiếu vừa phải
- Các biến còn lại có tỷ lệ thiếu rất thấp (<3%)

### 2.2. Tỉ lệ target cho trạng thái giá trị thiếu

In [ ]:
cols_to_compare = [col for col, summary in missing_summary.items() if 0 < summary['missing_percentage'] < 50]
viz.plot_target_rate_by_missing_grid(data, cols_to_compare)

**Nhận xét:**

Phân tích tỷ lệ `target=1` (tìm kiếm việc mới) giữa nhóm có và không có giá trị thiếu cho thấy những khác biệt đáng chú ý:

**Các biến có sự khác biệt lớn:**
- **company_type**: Nhóm có giá trị thiếu có tỷ lệ target=1 cao hơn đáng kể (0.39 so với 0.18), chênh lệch gấp hơn 2 lần. Điều này cho thấy việc thiếu thông tin về loại hình công ty có thể liên quan chặt chẽ đến xu hướng tìm việc mới.
- **company_size**: Tương tự, nhóm thiếu dữ liệu có tỷ lệ target=1 cao hơn (0.41 so với 0.18), thể hiện mối liên hệ mạnh mẽ giữa việc không cung cấp thông tin quy mô công ty và ý định chuyển việc.

**Các biến có sự khác biệt vừa phải:**
- **last_new_job**: Chênh lệch tương đối với tỷ lệ 0.36 so với 0.25 cho thấy những người không tiết lộ thời gian công việc gần nhất có xu hướng tìm việc cao hơn.
- **gender**: Chênh lệch tương đối (0.31 so với 0.23), cho thấy việc thiếu thông tin giới tính cũng có liên quan đến hành vi tìm việc.
- **enrolled_university**: Nhóm thiếu dữ liệu có tỷ lệ 0.32 so với 0.25, phản ánh khả năng những người không cung cấp thông tin học vấn có động cơ tìm việc mạnh hơn.
- **experience**: Tỷ lệ 0.35 so với 0.25 mặc dù số lượng quan sát thiếu rất nhỏ (n=65).

**Các biến có sự khác biệt nhỏ:**
- **education_level** và **major_discipline**: Chênh lệch không đáng kể giữa hai nhóm (0.23 vs 0.25 và 0.20 vs 0.26), cho thấy việc thiếu dữ liệu ở các biến này ít tác động đến biến mục tiêu.

**Kết luận:**
Bản thân việc thiếu dữ liệu, đặc biệt ở các biến liên quan đến thông tin công ty hiện tại (company_type, company_size) là một tín hiệu dự báo mạnh mẽ cho xu hướng tìm kiếm việc làm mới. 

# 3. Phân Tích Đơn Biến

### 3.1. Phân tích biến mục tiêu 

In [ ]:
target_dist = dp.get_target_distribution(data)
viz.plot_target_distribution(target_dist['counts'], target_dist['percentages'])

**Nhận xét:**

- Tập dữ liệu bị **mất cân bằng nghiêm trọng** với lớp `target=0.0` chiếm đa số (khoảng 75%) so với lớp `target=1.0` (khoảng 25%). Tỷ lệ này là khoảng 3:1. Điều này cần được xử lý trong giai đoạn mô hình hóa.
- Tập dữ liệu có **nhãn nhị phân**, hiện tại đang được biểu diễn là `0.0` và `1.0`, cần đổi về dạng `int` là `0` và `1`. 

### 3.2. Phân tích các biến numerical

In [ ]:
numerical_cols = [name for name, dtype in data.dtype.fields.items() if np.issubdtype(dtype[0], np.number)]
numerical_cols.remove('enrollee_id')
if 'target' in numerical_cols:
    numerical_cols.remove('target')
print(f"Các biến số ({len(numerical_cols)}): {numerical_cols}")

#### a. Phân phối và xu hướng trung tâm

In [ ]:
numerical_summary = dp.get_numerical_summary(data, numerical_cols)
for col, summary in numerical_summary.items():
    print(f"Thống kê mô tả cho cột '{col}':")
    for stat, value in summary.items():
        print(f"  {stat:<10} {value:>10.2f}")

In [ ]:
for col in numerical_cols:
    viz.plot_numerical_distribution_and_boxplot(data, col)

**Phân phối:**
- **`city_development_index`**: Phân phối multimodal với hai đỉnh rõ rệt tại ~0.62 và ~0.92. Đa số ứng viên (>70%) đến từ các thành phố có chỉ số phát triển cao (>0.9), phản ánh việc hầu hết các công ty tuyển dụng đều đến từ các đô thị lớn.
- **`training_hours`**: Phân phối lệch phải mạnh, đỉnh tập trung ở 20-40 giờ. Khoảng 70-80% ứng viên hoàn thành dưới 100 giờ đào tạo, cho thấy đa số chỉ tham gia các khóa học ngắn.

**Outliers:**
- **`city_development_index`**: Có một số outliers ở phía giá trị thấp (<0.5), đại diện cho các thành phố kém phát triển. Số lượng ít nên có thể giữ lại hoặc xử lý riêng tùy theo phân tích sâu hơn.
- **`training_hours`**: Có rất nhiều outliers ở phía trên (>~180 giờ), thể hiện nhóm ứng viên có cam kết cao với việc học.

#### b. Tương quan giữa biến numerical và target

In [ ]:
viz.plot_numerical_vs_target_grid(data, numerical_cols)

- **`city_development_index`**

  * Nhóm **target = 1** có **độ phân tán lớn hơn** (IQR rộng hơn) so với nhóm target = 0.
  * **Median của target = 1 thấp hơn đáng kể** so với target = 0.
  * Nhóm **target = 0** có phân phối tập trung hơn ở mức **cao hơn ~0.9**, trong khi nhóm target = 1 trải rộng từ thấp đến cao.

  **$\Rightarrow$ Ứng viên từ các thành phố kém phát triển hơn có xu hướng tìm việc (target = 1) cao hơn**, nhưng phân phối của họ cũng đa dạng hơn.

- **`training_hours`**

  * Hai nhóm target có **median gần như tương đương**.
  * Phân phối tổng thể giữa hai nhóm **gần như giống nhau**, cả về IQR và vị trí median.
  * Tuy nhiên, quantity outliers (giờ training rất cao) xuất hiện ở **cả hai nhóm** và khá nhiều.

  **$\Rightarrow$ `training_hours` không cho thấy sự khác biệt rõ rệt giữa hai nhóm**, có thể không phải biến phân biệt mạnh khi quan sát đơn biến.


#### c. Đánh giá sự khác biệt về thang đo

In [ ]:
viz.plot_scaling_comparison(data, col1='city_development_index', col2='training_hours')


**Miền giá trị:**
- **`city_development_index`**: Nằm trong khoảng rất hẹp từ 0.45 đến 0.95 (chênh lệch ~0.5)
- **`training_hours`**: Trải rộng từ 1 đến 336 giờ (chênh lệch 336 đơn vị)
- Tỷ lệ chênh lệch giữa hai biến là khoảng **1:672** (336/0.5)

**Vấn đề scaling:**
- Sự khác biệt thang đo rất lớn giữa hai biến sẽ gây ra vấn đề nghiêm trọng cho các thuật toán nhạy cảm với khoảng cách (KNN, SVM, K-Means) hoặc sử dụng gradient descent (Logistic Regression, Neural Networks)
- Nếu không scaling, `training_hours` sẽ **chi phối hoàn toàn** quá trình học của mô hình do giá trị số lớn hơn rất nhiều
- `city_development_index` có thể bị "bỏ qua" mặc dù là yếu tố dự báo quan trọng (như đã thấy ở phân tích trước)

**Quan sát thêm:**
- Dữ liệu tập trung theo các "cột dọc" tại một số giá trị city_development_index cố định (~0.62, ~0.92), phản ánh phân phối multimodal
- Không có mối tương quan tuyến tính rõ ràng giữa hai biến

**Kết luận:** 
Bắt buộc phải áp dụng **chuẩn hóa** cho tất cả các biến số trước khi huấn luyện mô hình để đảm bảo mỗi đặc trưng đóng góp công bằng vào quá trình dự báo.

#### d. Chất lượng dữ liệu

In [ ]:
print("Kiểm tra các giá trị không thể có:")
print(f"Số giờ đào tạo <= 0: {np.sum(data['training_hours'] <= 0)}")
print(f"Chỉ số phát triển thành phố < 0: {np.sum(data['city_development_index'] < 0)}")
print(f"Chỉ số phát triển thành phố > 1: {np.sum(data['city_development_index'] > 1)}")

**Nhận xét:**
Không có giá trị bất khả thi nào được tìm thấy trong các cột số. Dữ liệu có vẻ sạch về mặt này.

### 3.3. Phân tích các biến phân loại

In [ ]:
categorical_cols = [name for name, dtype in data.dtype.fields.items() if not np.issubdtype(dtype[0], np.number)]
print(f"Các biến phân loại ({len(categorical_cols)}): {categorical_cols}")

#### a. Chất lượng dữ liệu

In [ ]:
print("Kiểm tra tính nhất quán của các giá trị phân loại:")
for col in categorical_cols:
    unique_vals = np.unique(data[col])
    if col == 'city':
        # Chỉ in số giá trị vì cột city có quá nhiều giá trị
        print(f"- Cột '{col}' ({len(unique_vals)} giá trị khác)")
        continue
    print(f"- Cột '{col}' ({len(unique_vals)} giá trị khác nhau): {unique_vals}")

**Nhận xét:**

**Tính nhất quán của các biến phân loại:**
- **Không có lỗi chính tả hoặc biến thể:** Các giá trị trong từng cột đều nhất quán, không có trường hợp như 'Male'/'male' hoặc 'Graduate'/'graduate'
- **Giá trị rỗng (`''`)**: Xuất hiện ở 8 trong 10 biến phân loại, đại diện cho missing values đã được phân tích ở phần trước
- **Giá trị 'Other'**: Xuất hiện ở `gender`, `major_discipline`, và `company_type`, là các giá trị hợp lệ để nhóm các trường hợp không thuộc các category chính

**Các biến cần lưu ý:**
- **`experience`**: Có 23 giá trị khác nhau bao gồm cả số (1-20), khoảng ('<1', '>20'), và missing (''). Biến này có thể cần chuyển đổi sang dạng số hoặc ordinal encoding
- **`company_size`**: Có 9 giá trị với các khoảng không đồng nhất ('10/49', '50-99',..., '10000+'). Cần ordinal encoding hoặc nhóm lại
- **`last_new_job`**: Có 7 giá trị bao gồm số (1-4), khoảng ('>4'), và 'never'. Cần xử lý thứ tự tăng dần
- **`city`**: Có tới 123 giá trị khác nhau, chiếm gần 1% tổng số dòng. Cần cân nhắc sử dụng target encoding hoặc frequency encoding thay vì one-hot encoding

**Kết luận:** Chất lượng dữ liệu của các biến phân loại khá tốt, không có lỗi nhập liệu rõ ràng. Tuy nhiên, cần chuyển đổi một số biến ordinal và xử lý high-cardinality cho biến `city`.

#### b. Phân phối giá trị

##### city

Do cột `city` có **123 giá trị khác nhau** (cardinality cao) nên phân tích chi tiết cho cột này sẽ được thực hiện riêng nếu cần thiết, vì trực quan hóa tất cả các thành phố sẽ không hiệu quả.

##### gender

In [ ]:
viz.plot_categorical_analysis(data, 'gender')

**Nhận xét:**

**Phân phối gender:**
- **Male** chiếm tuyệt đại đa số với 13,221 ứng viên (~69%), trong khi Female chỉ có 1,238 (~6.5%) và Other 191 (~1%)
- Có 4,508 giá trị missing (~23.5%), đã được phân tích ở phần trước
- Dữ liệu có sự **mất cân bằng nghiêm trọng** về giới tính, phản ánh thực tế ngành Data Science đang bị nam giới thống trị

**Tỷ lệ target theo gender:**
- **Missing values** có tỷ lệ target=1 cao nhất (0.31), cao hơn mức trung bình
- **Male** có tỷ lệ dưới mức trung bình, trong khi **Female và Other** có tỷ lệ tương đương nhau và gần với mức trung bình
- Sự khác biệt giữa các nhóm không quá lớn (0.23-0.31), cho thấy gender có **ảnh hưởng vừa phải** đến quyết định chuyển việc

**Insight:** Việc thiếu thông tin gender lại có tỷ lệ target=1 cao hơn, tương tự pattern đã thấy ở các biến liên quan đến công ty, củng cố giả thuyết về nhóm ứng viên "cố tình không tiết lộ thông tin cá nhân" có xu hướng chuyển việc cao hơn.

##### relevent_experience

In [ ]:
viz.plot_categorical_analysis(data, 'relevent_experience')

**Nhận xét:**

**Phân phối relevant_experience:**
- **Has relevant experience** chiếm đa số tuyệt đối với 13,792 ứng viên (~72%)
- **No relevant experience** chỉ có 5,366 ứng viên (~28%)
- Tỷ lệ 72:28 cho thấy hầu hết ứng viên tham gia khóa đào tạo đã có nền tảng kinh nghiệm liên quan đến Data Science

**Tỷ lệ target theo relevant_experience:**
- **No relevant experience** có tỷ lệ target=1 **cao hơn đáng kể** (0.34 vs 0.21), cao hơn mức trung bình (~0.25)
- Chênh lệch lớn (~62% cao hơn: 0.34/0.21) cho thấy đây là **yếu tố dự báo quan trọng**

**Insight:**
- Người **không có kinh nghiệm** liên quan có xu hướng tìm việc mới cao hơn, có thể vì:
  - Đang muốn chuyển đổi nghề nghiệp hoàn toàn → cần công việc mới để áp dụng kiến thức
  - Chưa có công việc ổn định trong lĩnh vực → đang tích cực tìm kiếm cơ hội
- Người **có kinh nghiệm** có thể chỉ muốn nâng cấp kỹ năng để phát triển ở vị trí hiện tại

**Kết luận:** `relevant_experience` là biến quan trọng, nhóm "No relevant experience" là target audience có khả năng rời công ty cao hơn sau đào tạo.

##### enrolled_university

In [ ]:
viz.plot_categorical_analysis(data, 'enrolled_university')

**Nhận xét:**

**Phân phối enrolled_university:**
- **no_enrollment** chiếm đa số với 13,817 ứng viên (~72%), cho thấy hầu hết ứng viên không đang theo học đại học.
- Phân phối bị **mất cân bằng** nghiêm trọng, nghiêng hẳn về phía `no_enrollment` cho thấy phần lớn là người không học đại học.

**Tỷ lệ target theo enrolled_university:**
- **Full time course** có tỷ lệ target=1 **cao nhất** (0.38), cao hơn đáng kể so với mức trung bình (~0.25)
- **Missing values** có tỷ lệ 0.32, cũng cao hơn trung bình
- **Part time course** có tỷ lệ bằng mức trung bình (0.25)
- **no_enrollment** có tỷ lệ **thấp nhất** (0.21), dưới mức trung bình

**Insight:**
- Sinh viên **Full time** có xu hướng tìm việc mới cao nhất, có thể vì:
  - Sắp tốt nghiệp và đang tích cực tìm việc đầu tiên
  - Đang học để chuyển đổi nghề nghiệp hoàn toàn
  - Chưa có công việc ổn định nên cần tìm việc sau đào tạo
- Người **không học đại học** (no_enrollment) có xu hướng ở lại công ty cao hơn, có thể đã có công việc ổn định và chỉ muốn nâng cao kỹ năng

**Kết luận:** `enrolled_university` là yếu tố dự báo quan trọng. Nhóm "Full time course" có rủi ro rời công ty cao nhất sau đào tạo.

##### education_level

In [ ]:
viz.plot_categorical_analysis(data, 'education_level')

**Nhận xét:**

**Phân phối education_level:**
- **Graduate** chiếm đa số áp đảo với 11,598 ứng viên (~60.5%), cho thấy hầu hết ứng viên có bằng đại học. 
- Phân phối bị **mất cân bằng** nghiêm trọng, nghiêng hẳn về phía **Graduate** cho thấy phần lớn là người đã tốt nghiệp đại học.

**Tỷ lệ target theo education_level:**

Có xu hướng **giảm dần** khi trình độ học vấn tăng:
  - **Graduate**: 0.28 (cao hơn mức trung bình ~0.25)
  - **Masters**: 0.21 (gần hoặc dưới mức trung bình)
  - **High School**: 0.20
  - **Phd**: 0.14 (thấp nhất, chỉ bằng một nửa Graduate)
  - **Primary School**: 0.13 (thấp nhất cùng Phd)

**Insight:**
- Người có bằng **Graduate** có xu hướng tìm việc mới cao nhất, có thể vì:
  - Đây là trình độ phổ biến nhất trong ngành Data Science → nhiều cơ hội việc làm
  - Đủ năng lực để di chuyển nhưng chưa quá "cao cấp" như Masters/Phd
- Người có **Phd/Primary School** ít tìm việc hơn, có thể vì:
  - **Phd**: Đã có vị trí ổn định trong nghiên cứu/học thuật, khó tìm việc phù hợp
  - **Primary School**: Giới hạn về trình độ, ít cơ hội chuyển việc

**Kết luận:** `education_level` có mối quan hệ **phi tuyến** với target. Nhóm Graduate có rủi ro rời công ty cao nhất, trong khi Phd thấp nhất.

##### major_discipline

In [ ]:
viz.plot_categorical_analysis(data, 'major_discipline')

**Nhận xét:**

**Phân phối major_discipline:**
- **STEM** chiếm đa số áp đảo với 14,492 ứng viên (~75.6%), phản ánh đúng bản chất của lĩnh vực Data Science
- Phân phối bị **mất cân bằng** nghiêm trọng, nghiêng hẳn về phía **STEM** cho thấy phần lớn các ứng viên học về lĩnh vực này.

**Tỷ lệ target theo major_discipline:**
- **Other**, **Business Degree** và **STEM** có tỷ lệ target=1 cao nhất (0.27 và 0.26), cao hơn mức trung bình
- Còn lại đều dưới mức trung bình

**Insight:**
- Người từ chuyên ngành **không phải STEM** (Other, Business) có xu hướng tìm việc cao hơn, có thể vì:
  - Đang chuyển đổi nghề nghiệp hoàn toàn → cần công việc mới để áp dụng
  - Chưa có nền tảng kỹ thuật vững → cần tìm việc entry-level
- Người **STEM** có tỷ lệ trung bình, có thể đã có việc ổn định trong lĩnh vực
- **Humanities/Arts** có tỷ lệ thấp bất ngờ, có thể do số lượng mẫu nhỏ hoặc họ tham gia khóa học vì sở thích hơn là chuyển nghề

**Kết luận:** `major_discipline` có ảnh hưởng vừa phải đến target. Nhóm chuyển đổi từ ngành khác (Other, Business) có xu hướng tìm việc cao hơn STEM.

##### experience

In [ ]:
viz.plot_categorical_analysis(data, 'experience')

**Nhận xét:**

**Phân phối experience:**
- **>20 năm** chiếm nhiều nhất với 3,286 ứng viên (~17.1%)
- Phân phối tương đối **đồng đều** qua các khoảng kinh nghiệm từ 1-10 năm (mỗi nhóm ~800-1,400 người)
- **<1 năm**: 522 ứng viên (~2.7%)
- **Missing**: 65 ứng viên (~0.34%) - tỷ lệ thiếu rất thấp
- Có 23 giá trị duy nhất, cho thấy biến này rất chi tiết

**Tỷ lệ target theo experience:**

Có xu hướng **giảm dần** khi kinh nghiệm tăng:
  - **<1 năm và 1 năm**: Tỷ lệ cao nhất (0.45 và 0.42), cao hơn đáng kể so với mức trung bình (~0.25)
  - **2-7 năm**: Tỷ lệ từ 0.29-0.35, cao hơn hoặc gần mức trung bình
  - **8-15 năm**: Tỷ lệ giảm xuống 0.17-0.24, dưới mức trung bình
  - **>15 năm**: Tỷ lệ thấp nhất (0.14-0.17), chỉ bằng ~1/3 nhóm junior

**Insight:**
- **Junior (<3 năm kinh nghiệm)** có xu hướng tìm việc mới cao nhất, có thể vì:
  - Đang trong giai đoạn khám phá nghề nghiệp, sẵn sàng thay đổi
  - Chưa gắn bó lâu dài với công ty hiện tại
  - Muốn tìm cơ hội tốt hơn sau khi nâng cấp kỹ năng
- **Senior (>15 năm)** có xu hướng ở lại thấp nhất, có thể vì:
  - Đã có vị trí và mức lương ổn định
  - Khó tìm việc phù hợp với level cao
  - Chi phí cơ hội khi chuyển việc lớn hơn

**Kết luận:** `experience` là yếu tố dự báo **rất quan trọng** với mối quan hệ **phi tuyến rõ rệt**. Nhóm junior (<3 năm) có rủi ro rời công ty cao gấp ~3 lần nhóm senior (>15 năm).

##### company_size

In [ ]:
viz.plot_categorical_analysis(data, 'company_size')

**Nhận xét:**

**Phân phối company_size:**
- **Missing** chiếm đa số với 5,938 ứng viên (~31%), phản ánh tỷ lệ thiếu cao đã phân tích trước
- Các quy mô khác phân bố  không cách biệt quá mạnh 

**Tỷ lệ target theo company_size:**
- **Missing** có tỷ lệ target=1 **cao nhất** (0.41), cao hơn rất nhiều so với mức trung bình (~0.25) - củng cố phát hiện ở phân tích missing values
- Còn lại đều dưới mức trung bình

**Insight:**
- `Missing` $\rightarrow$ Tỷ lệ tìm việc cao nhất, có thể là:
  - Sinh viên/người thất nghiệp chưa có công ty
  - Freelancer/tự kinh doanh
  - Cố tình không tiết lộ vì đang muốn rời bỏ công ty hiện tại
- **Công ty nhỏ (10-49 người)** → Tỷ lệ cao thứ hai, có thể vì:
  - Lương thấp, phúc lợi kém hơn công ty lớn
  - Cơ hội phát triển hạn chế
  - Ổn định tài chính kém
- **Công ty lớn/trung bình** → Nhân viên ổn định hơn, ít muốn chuyển việc

**Kết luận:** `company_size` là yếu tố dự báo **rất quan trọng**. Missing value và công ty nhỏ có rủi ro rời bỏ cao gấp ~2-3 lần so với công ty lớn. Cần tạo missing indicator feature.

##### company_type

In [ ]:
viz.plot_categorical_analysis(data, 'company_type')

**Nhận xét:**

**Phân phối company_type:**
- **Pvt Ltd** (công ty tư nhân) chiếm đa số với 9,817 ứng viên (~51.2%)
- **Missing** có 6,140 ứng viên (~32%), tỷ lệ thiếu cao như đã phân tích
- Còn lại có phân phối khá đều, cho thấy sự **mất cân bằng** nghiêm trọng, nghiêng hẳn về phía `Pvt Ltd` và `Missing`

**Tỷ lệ target theo company_type:**
- **Missing** có tỷ lệ target=1 **cao nhất** (0.39), cao hơn rất nhiều so với mức trung bình (~0.25) - pattern tương tự company_size
- Còn lại đều dưới mức trung bình

**Insight:**
- **Missing** → Tỷ lệ tìm việc cao nhất, củng cố giả thuyết về nhóm không có công ty hoặc cố tình giấu thông tin
- **Early Stage Startup** → Tỷ lệ cao, có thể vì:
  - Công ty không ổn định, rủi ro phá sản
  - Lương thấp, áp lực cao
  - Muốn chuyển sang môi trường tốt hơn
- **Funded Startup** → Tỷ lệ thấp nhất, có thể vì:
  - Đã có funding nên ổn định hơn Early Stage
  - Lương và quyền chọn mua cổ phần hấp dẫn
  - Môi trường năng động, cơ hội thăng tiến
- **Pvt Ltd** (chiếm >50%) → Tỷ lệ thấp, phản ánh sự ổn định của các công ty tư nhân lớn

**Kết luận:** `company_type` là yếu tố dự báo **rất quan trọng**. Missing và Early Stage Startup có rủi ro cao gấp ~3 lần Funded Startup. Cần tạo missing indicator feature và xem xét group các startup lại.

##### last_new_job

In [ ]:
viz.plot_categorical_analysis(data, 'last_new_job')

**Nhận xét:**

**Phân phối last_new_job:**
- **1 năm** chiếm nhiều nhất với 8,040 ứng viên (~42%), cho thấy đa số ứng viên vừa mới chuyển công việc gần đây
- **>4 năm**, **2 năm** và **never** tương đối ngang nhau
- **4 năm**, **3 năm** và **Missing** thuộc nhóm thấp nhất

**Tỷ lệ target theo last_new_job:**
- **Missing** có tỷ lệ target=1 **cao nhất** (0.36), cao hơn đáng kể so với mức trung bình (~0.25)
- **never** và **1 năm**: đều cao hơn mức trung bình
- Còn lại đều dưới mức trung bình

**Insight:**
- **Missing và "never"** → Tỷ lệ tìm việc cao, có thể vì:
  - Missing: Sinh viên mới ra trường chưa có lịch sử việc làm
  - Never: Đang ở công việc đầu tiên, đã sẵn sàng thử thách mới sau đào tạo
- **1-2 năm** (vừa chuyển việc gần đây) → Tỷ lệ trung bình, có thể:
  - Vẫn đang thích nghi với môi trường mới
  - Nhưng cũng sẵn sàng xem xét cơ hội tốt hơn
- **>4 năm** (gắn bó lâu dài) → Tỷ lệ thấp nhất, phản ánh:
  - Đã ổn định và hài lòng với công việc hiện tại
  - Chi phí cơ hội khi chuyển việc cao
  - Có thể đã lên vị trí cao, khó tìm việc tương đương

**Kết luận:** `last_new_job` là yếu tố dự báo **quan trọng** với xu hướng rõ ràng: Người chưa từng chuyển việc hoặc vừa chuyển gần đây có xu hướng tìm việc cao gấp ~2 lần người gắn bó >4 năm. Biến này có tính ordinal cần xử lý đúng thứ tự.

### 3.4. Nhận xét tổng quan về các đặc trưng

Phân tích đơn biến đã cung cấp nhiều insight giá trị về các yếu tố ảnh hưởng đến xu hướng tìm việc của ứng viên:

1.  **Các yếu tố dự báo mạnh nhất:**
    *   **Kinh nghiệm (`experience`, `last_new_job`):** Đây là nhóm yếu tố có ảnh hưởng rõ rệt nhất. Ứng viên có ít kinh nghiệm (đặc biệt là dưới 5 năm) hoặc chưa bao giờ chuyển việc (`never`) có xu hướng tìm việc mới cao hơn đáng kể. Ngược lại, những người có kinh nghiệm lâu năm (>15 năm) hoặc đã gắn bó với công việc hiện tại trên 4 năm rất ổn định và ít có ý định thay đổi.
    *   **Thông tin công ty (`company_size`, `company_type`):** Việc thiếu thông tin về công ty là một tín hiệu dự báo cực kỳ mạnh. Ngoài ra, nhân viên làm việc tại các công ty nhỏ hoặc startup giai đoạn đầu (`Early Stage Startup`) cũng có tỷ lệ tìm việc cao hơn hẳn so với các công ty lớn, công ty tư nhân (`Pvt Ltd`) hoặc startup đã có vốn (`Funded Startup`).
    *   **Tình trạng học vấn (`enrolled_university`):** Sinh viên đang theo học toàn thời gian (`Full time course`) có tỷ lệ tìm việc cao nhất, cho thấy họ đang chuẩn bị cho sự nghiệp sau khi tốt nghiệp hoặc muốn chuyển ngành.

2.  **Các yếu tố dự báo có ảnh hưởng vừa phải:**
    *   **Chỉ số phát triển thành phố (`city_development_index`):** Ứng viên đến từ các thành phố có chỉ số phát triển thấp hơn có xu hướng tìm việc cao hơn. Đây là một yếu tố có giá trị dự báo.
    *   **Kinh nghiệm liên quan (`relevent_experience`):** Những người không có kinh nghiệm liên quan có xu hướng tìm việc cao hơn, có thể vì họ đang muốn chuyển sang một lĩnh vực mới.
    *   **Trình độ học vấn (`education_level`):** Mối quan hệ không hoàn toàn tuyến tính. Nhóm `Graduate` có tỷ lệ tìm việc cao nhất, trong khi nhóm `Masters` và đặc biệt là `Phd` lại có xu hướng ổn định hơn.
    *   **Giới tính (`gender`) và Chuyên ngành (`major_discipline`):** Có sự khác biệt nhưng không quá lớn. Đáng chú ý là việc thiếu thông tin giới tính lại tương quan với tỷ lệ tìm việc cao hơn.

3.  **Các yếu tố có ảnh hưởng yếu hoặc không rõ ràng:**
    *   **Số giờ đào tạo (`training_hours`):** Biến này không cho thấy sự khác biệt rõ rệt giữa hai nhóm `target`. Phân phối của nó gần như giống hệt nhau bất kể ứng viên có tìm việc hay không. Do đó, nó có thể không phải là một yếu tố dự báo mạnh khi đứng một mình.

**Kết luận chung:** Chân dung một ứng viên có khả năng cao đang tìm kiếm công việc mới là người có ít kinh nghiệm, đến từ một thành phố có chỉ số phát triển thấp, đang làm ở một công ty nhỏ/startup hoặc không tiết lộ thông tin công ty, và có thể đang là sinh viên toàn thời gian. Ngược lại, một ứng viên ổn định thường là người có nhiều năm kinh nghiệm, đến từ các thành phố phát triển và làm việc cho một công ty tư nhân lớn hoặc startup đã có vốn.

# 4. Phân Tích Đa Biến

### 4.1. Mối quan hệ và tương quan

#### Tương quan giữa các biến số

In [ ]:
viz.plot_correlation_heatmap(data, numerical_cols)

**Nhận xét:**

Heatmap tương quan cho thấy không có mối tương quan tuyến tính mạnh giữa `city_development_index` và `training_hours`. Điều này cho thấy chúng cung cấp thông tin độc lập cho mô hình.

### 4.2. Bảng Chéo (Cross-tabulations)

#### Biến phân loại vs. Biến phân loại

In [ ]:
# Lấy các giá trị duy nhất cho mỗi chiều
edu_levels = np.unique(data['education_level'])
exp_levels = np.unique(data['relevent_experience'])

# Tạo một ma trận rỗng để lưu số đếm
crosstab_counts = np.zeros((len(edu_levels), len(exp_levels)), dtype=int)

# Điền vào ma trận
for i, edu in enumerate(edu_levels):
    for j, exp in enumerate(exp_levels):
        # Tạo một mặt nạ boolean để đếm các lần xuất hiện
        mask = (data['education_level'] == edu) & (data['relevent_experience'] == exp)
        crosstab_counts[i, j] = np.sum(mask)

# Trực quan hóa bảng chéo bằng heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(crosstab_counts, annot=True, fmt='d', cmap='YlGnBu', 
            xticklabels=exp_levels, yticklabels=edu_levels)
plt.title('Heatmap: Education Level vs. Relevant Experience')
plt.ylabel('Education Level')
plt.xlabel('Relevant Experience')
plt.show()

**Nhận xét:**
Heatmap cho thấy phần lớn ứng viên có kinh nghiệm liên quan (`Has relevent experience`) là những người đã tốt nghiệp đại học (`Graduate`). Điều này là một mối quan hệ hợp lý và được mong đợi.

#### Biến số vs. Biến phân loại

In [ ]:
def print_summary_table(summary, num_col, cat_col):
    print(f"Thống kê mô tả cho '{num_col}' theo '{cat_col}':")
    # Print header
    print(f"{'Category':<25} | {'Count':>10} | {'Mean':>10} | {'Std Dev':>10} | {'Median':>10}")
    print('-' * 75)
    # Print rows
    for category, stats in summary.items():
        print(f"{category:<25} | {stats['count']:>10} | {stats['mean']:>10.2f} | {stats['std']:>10.2f} | {stats['median']:>10.2f}")
    print('\n')

# Cặp 1: city_development_index vs. relevent_experience
summary1 = dp.get_numerical_summary_by_category(data, 'city_development_index', 'relevent_experience')
print_summary_table(summary1, 'city_development_index', 'relevent_experience')

# Cặp 2: training_hours vs. education_level
summary2 = dp.get_numerical_summary_by_category(data, 'training_hours', 'education_level')
print_summary_table(summary2, 'training_hours', 'education_level')

**Nhận xét:**

*   **`city_development_index` vs. `relevent_experience`**: Những người có kinh nghiệm liên quan (`Has relevent experience`) có xu hướng đến từ các thành phố có chỉ số phát triển cao hơn một chút (trung bình 0.85 so với 0.78). Điều này hợp lý vì các thành phố phát triển hơn thường cung cấp nhiều cơ hội việc làm trong ngành hơn.
*   **`training_hours` vs. `education_level`**: Không có sự khác biệt lớn về số giờ đào tạo trung bình giữa các trình độ học vấn, ngoại trừ nhóm `Primary School` có số giờ đào tạo trung bình cao hơn một cách đáng ngạc nhiên. Tuy nhiên, số lượng mẫu của nhóm này rất nhỏ, vì vậy kết quả này có thể không đáng tin cậy.

# 5. Quan Sát Ban Đầu & Insights

### Tóm tắt các quan sát chính

1.  **Việc thiếu dữ liệu là một tín hiệu mạnh:** Việc không cung cấp thông tin về công ty (`company_size`, `company_type`) có mối tương quan mạnh với việc một người đang tìm kiếm công việc mới.
2.  **Kinh nghiệm là yếu tố quyết định:** Kinh nghiệm làm việc càng ít, xu hướng tìm việc mới càng cao. Những người có dưới 5 năm kinh nghiệm đặc biệt có khả năng "nhảy việc" cao.
3.  **Môi trường làm việc quan trọng:** Nhân viên tại các công ty nhỏ và startup có tỷ lệ tìm việc mới cao hơn đáng kể so với các công ty lớn và ổn định hơn.
4.  **Vị trí địa lý có ảnh hưởng:** Ứng viên từ các thành phố có chỉ số phát triển thấp hơn có nhiều khả năng tìm kiếm cơ hội mới hơn.
5.  **Dữ liệu bị mất cân bằng:** Lớp `target=1` (tìm việc) chỉ chiếm khoảng 25% tập dữ liệu, điều này cần được xử lý trong giai đoạn mô hình hóa.

### Các vấn đề về chất lượng dữ liệu

- **Giá trị thiếu:** Nhiều cột có tỷ lệ thiếu đáng kể, cần có chiến lược xử lý thông minh (ví dụ: coi là một loại riêng) thay vì chỉ điền giá trị.
- **Cardinality cao:** Cột `city` có quá nhiều giá trị duy nhất, cần một chiến lược mã hóa đặc biệt (ví dụ: target encoding, frequency encoding) thay vì one-hot encoding.
- **Thang đo khác biệt:** Các biến số `city_development_index` và `training_hours` có thang đo rất khác nhau và cần được chuẩn hóa (scaling).

### Các bước tiền xử lý cần thiết

1.  **Xử lý giá trị thiếu:** Chuyển đổi các giá trị thiếu trong các cột phân loại thành một loại mới (ví dụ: 'Missing').
2.  **Mã hóa biến phân loại:**
    - Sử dụng One-Hot Encoding cho các biến có ít loại.
    - Sử dụng Target Encoding hoặc Frequency Encoding cho cột `city`.
    - Chuyển đổi các biến thứ tự (ordinal) như `experience` và `education_level` thành số.
3.  **Chuẩn hóa biến số:** Áp dụng `StandardScaler` hoặc `MinMaxScaler` cho các biến số.
4.  **Xử lý mất cân bằng:** Áp dụng SMOTE cho tập huấn luyện hoặc sử dụng `class_weight` trong mô hình.

### Cờ đỏ (Red Flags) & Hạn chế

- **Định nghĩa "tìm việc":** Dữ liệu không phân biệt giữa người đang tích cực tìm việc và người chỉ "thụ động" xem xét các cơ hội. Điều này có thể ảnh hưởng đến độ chính xác của mô hình.
- **Thiên vị lựa chọn (Selection Bias):** Dữ liệu chỉ bao gồm những người đã tham gia một khóa đào tạo. Kết quả có thể không khái quát hóa được cho toàn bộ lực lượng lao động.